In [1]:
class KnowledgeBase:
    """stores and manages our user profiles by assigning unique identifiers to each of the users"""
    def __init__(self):  # Empty dictionary storing user profiles
        self.profiles = {}
        self.next_id = 1  # Auto-increments ID

    def add_profile(self, profile_data):
        profile_id = self.next_id
        self.profiles[profile_id] = {'id': profile_id, **profile_data}  # add profile data with ID
        self.next_id += 1
        return profile_id

    def remove_profile(self, profile_id):  # removes a profile from the knowledge base
        if profile_id in self.profiles:
            del self.profiles[profile_id]

    def get_profile(self, profile_id):  # retrieves a profile by ID
        return self.profiles.get(profile_id)

    def get_all_profiles(self):  # retrieves all profiles
        return list(self.profiles.values())


class Matchmaker:
    """Implements a matchmaking system logic using compatibility rules and weights
    leverages the KB to find optimal matches for a given profile, considering Tinder preferences"""

    def __init__(self, knowledge_base, weights=None, max_age_diff=20, preferred_sex=None, height=None, relationship_type=None, love_language=None, 
                 personality=None, mbti=None, religion=None, sexual_orientation=None):
        """
        :param knowledge_base: The knowledge base to retrieve profiles from.
        :param weights: Dictionary containing attributes weights for matchmaking (sum to 1).
        :param max_age_diff: Maximum age difference allowed for profiles.
        :param preferred_sex: Preferred sex for matching (can be 'male', 'female', or 'non-binary').
        :param relationship_type: Type of relationship the user is seeking ('casual', 'serious', etc.).
        :param love_language: Type of love_language('words of affirmation', 'physical touch', 'acts of service', 'gift giving', 'quality type')
        :param personality_type('Introvert', 'extrovert', 'ambivert')
        :param mbti_type('ISTJ', 'ISFJ', 'INFJ', 'INTJ', 'ISTP', 'ISFP', 'INFP', 'INTP', 'ESTP', 'ESFP', 'ENFP', 'ENTP', 'ESTJ', 'ESFJ', 'ENFJ', 'ENTJ')
        :param religion('Christian', 'Muslim', 'Buddhist', 'Hindu', 'Jewish', 'Sikh', 'Atheist')
        :param sexual_orientation('heterosexual', 'homosexual', 'bisexual', 'asexual')
        """
        self.knowledge_base = knowledge_base
        self.weights = weights or {'age': 0.25, 'interests': 0.5, 'location': 0.15, 'sex': 0.1, 'relationship': 0.1,'love_langauage':0.5,
                                   'personality':0.1,'mbti':0.15,'religion':0.1,'sexual_orientation':0.1}
        self.max_age_diff = max_age_diff
        self.preferred_sex = preferred_sex
        self.relationship_type = relationship_type
        self.love_language = love_language
        self.personality = personality
        self.mbti= mbti
        self.religion = religion
        self.sexual_orientation = sexual_orientation

    def calculate_compatibility(self, profile_a, profile_b):
        """computes the compatibility score between two profiles
           implements domain-specific rules and weights"""

        # Age Compatibility (Normalized inverse difference)
        age_diff = abs(profile_a['age'] - profile_b['age'])  # absolute age difference
        age_score = max(0, 1 - (age_diff / self.max_age_diff)) * self.weights['age']


        #Personality Compatibility
        personality_score = 0
        if self.personality:
            if profile_b['personality'] == self.personality:
                personality_score = self.weights['personality']


        #Love language Compatibility
        love_language_score = 0
        if self.love_language:
            if profile_b['love_language'] == self.love_language:
                love_language_score = self.weights['love_language']


        #MBTI Compatibility
        mbti_score = 0
        if self.mbti:
            if profile_b['mbti'] == self.mbti:
                mbti_score = self.weights['mbti']


        #Sexual Orientation Compatibility
        sexual_orientation_score = 0
        if self.sexual_orientation:
            if profile_b['sexual_orientation'] == self.sexual_orientation:
                sexual_orientation_score = self.weights['sexual_orientation']


        #Religion Compatibility
        religion_score = 0
        if self.religion:
            if profile_b['religion'] == self.religion:
                religion_score = self.weights['religion']


        # Interests Compatibility (Jaccard Similarity)
        interests_a = set(profile_a['interests'])
        interests_b = set(profile_b['interests'])
        intersection = len(interests_a & interests_b)  # common_interests
        union = len(interests_a | interests_b)
        jaccard_score = (intersection / union if union else 0) * self.weights['interests']

        # Location Compatibility
        location_score = self.weights['location'] if profile_a['location'] == profile_b['location'] else 0

        # sex Compatibility
        sex_score = 0
        if self.preferred_sex:
            if profile_b['sex'] == self.preferred_sex:
                sex_score = self.weights['gender']

        # Relationship Type Compatibility
        relationship_score = 0
        if self.relationship_type:
            if profile_b['relationship_type'] == self.relationship_type:
                relationship_score = self.weights['relationship']

        # Total Compatibility Score
        return age_score + jaccard_score + location_score + sex_score + relationship_score+ personality_score + love_language_score + mbti_score + sexual_orientation_score + religion_score

    def find_matches(self, user_id, top_n=5):
        """Finds the top N matches for a given user based on compatibility score
           :param user_id: Profile ID of the user to find matches for
           :param top_n: Number of top matches to return
           :return: List of (profile_id, compatibility_score) tuples"""

        target_profile = self.knowledge_base.get_profile(user_id)
        if not target_profile:
            return []

        matches = []
        for profile in self.knowledge_base.get_all_profiles():
            if profile['id'] == user_id:
                continue  # Skip the same profile

            # Calculate compatibility score for each profile
            score = self.calculate_compatibility(target_profile, profile)
            matches.append((profile['id'], round(score, 2)))

        # Sort matches by score and return top N
        matches.sort(key=lambda x: x[1], reverse=True)  # Sort by compatibility score
        return matches[:top_n]


# Test the matchmaking system with profiles and preferences
if __name__ == '__main__':
    # Initialize the Knowledge Base
    kb = KnowledgeBase()

    # Add sample profiles with gender and relationship type
    profiles = [
        {'age': 25, 'interests': ['music', 'sports'], 'location': 'New York', 'sex': 'female', 'relationship_type': 'casual', 'personality': 
         'introvert', 'love_language':'acts_of_service','mbti':'INTP', 'sexual_orientation':'bisexual','religion': 'christian'},
        {'age': 21, 'interests': ['movies', 'politics'], 'location': 'Detroit', 'sex': 'male', 'relationship_type': 'serious', 'personality':
          'extrovert', 'love_language':'gift_giving','mbti':'INTP', 'sexual_orientation':'asexual','religion': 'muslim'},
        {'age': 26, 'interests': ['anime', 'wrestling'], 'location': 'Los Angeles', 'sex': 'female', 'relationship_type': 'casual', 'personality':
          'extrovert', 'love_language':'acts_of_service','mbti':'INTP', 'sexual_orientation':'homosexual','religion': 'hindu'},
        {'age': 22, 'interests': ['hiking', 'road_trips'], 'location': 'Wyoming', 'sex': 'male', 'relationship_type': 'serious', 'personality':
          'introvert', 'love_language':'physical_touch','mbti':'INTP', 'sexual_orientation':'heterosexual','religion': 'buddhist'},
        {'age': 20, 'interests': ['vlogging', 'travel'], 'location': 'Idaho', 'sex': 'female', 'relationship_type': 'casual', 'personality': 
         'introvert', 'love_language':'quality_time','mbti':'INTP', 'sexual_orientation':'homosexual','religion': 'christian'},
        {'age': 19, 'interests': ['volunteering', 'camping'], 'location': 'DC Washington', 'sex': 'male', 'relationship_type': 'serious', 'personality':
          'extrovert', 'love_language':'words_of_affirmation','mbti':'INTP', 'sexual_orientation':'bisexual','religion': 'Jewish'},
        {'age': 28, 'interests': ['music', 'politics'], 'location': 'Tennessee', 'sex': 'female', 'relationship_type': 'casual', 'personality':
          'introvert', 'love_language':'acts_of_service','mbti':'INTP', 'sexual_orientation':'heterosexual','religion': 'christian'},
        {'age': 31, 'interests': ['skating', 'reading'], 'location': 'Seattle', 'sex': 'male', 'relationship_type': 'serious', 'personality':
          'extrovert', 'love_language':'acts_of_service','mbti':'INTP', 'sexual_orientation':'homosexual','religion': 'sikh'},
        {'age': 35, 'interests': ['music', 'sports'], 'location': 'Arkansas', 'sex': 'female', 'relationship_type': 'casual', 'personality':
          'introvert', 'love_language':'acts_of_service','mbti':'INTP', 'sexual_orientation':'bisexual','religion': 'atheist'},
        {'age': 40, 'interests': ['reading', 'music'], 'location': 'Texas', 'sex': 'male', 'relationship_type': 'serious', 'personality':
          'extrovert', 'love_language':'acts_of_service','mbti':'INTP', 'sexual_orientation':'heterosexual','religion': 'christian'},
        {'age': 24, 'interests': ['archery', 'singing'], 'location': 'Clifornia', 'sex': 'male', 'relationship_type': 'casual', 'personality':
          'introvert', 'love_language':'acts_of_service','mbti':'INTP', 'sexual_orientation':'bisexual','religion': 'Jewish'},
        {'age': 23, 'interests': ['music', 'dancing'], 'location': 'Alabama', 'sex': 'female', 'relationship_type': 'serious', 'personality':
          'extrovert', 'love_language':'acts_of_service','mbti':'INTP', 'sexual_orientation':'homosexual','religion': 'muslim'},
    ]

    for p in profiles:
        kb.add_profile(p)

    # Initialize the Matchmaker with Tinder preferences (e.g., prefer females and serious relationships)
    matchmaker =Matchmaker(
        knowledge_base=kb,
        weights={'age': 0.25, 'interests': 0.5, 'location': 0.15, 'gender': 0.1, 'relationship': 0.1,'love_language':0.5,'personality':0.1,'mbti':0.15,
                 'religion':0.1,'sexual_orientation':0.1},
        max_age_diff=25,
        preferred_sex='female',
        relationship_type='serious',
        personality='introvert',
        love_language='acts_of_service',
        mbti='INTP',
        sexual_orientation='bisexual',
        religion='christian'
    )

    # Find matches for the first profile (ID 1) and print the top 3 matches
    print("Top matches for profile 1:")
    for match in matchmaker.find_matches(1, top_n=3):
        print(f"Profile {match[0]}, Compatibility Score: {match[1]}")


Top matches for profile 1:
Profile 9, Compatibility Score: 1.6
Profile 7, Compatibility Score: 1.34
Profile 12, Compatibility Score: 1.25
